### Environment

In [ ]:
%cd external/YOLOX/
%pip install -r requirements.txt && python setup.py develop

%cd ../external/deep-person-reid/
%pip install -r requirements.txt && python setup.py develop

%cd ../external/fast_reid/
%pip install -r docs/requirements.txt


In [ ]:
%mv /home/Deep-OC-SORT/external/weights/* /home/share/model/DeepOC-SORT/

### Data Preparation

In [ ]:
!unzip /home/share/data_warehouse/MOTChallenge/MOT17-FRCNN.zip -d /home/share/datasets/mot
!unzip /home/share/data_warehouse/MOTChallenge/MOT20.zip -d /home/share/datasets

In [ ]:
# 转为COCO格式
# replace "dance" with mot17/mot20 for others
!python data/tools/convert_dance_to_coco.py 

### Evaluate

In [ ]:
%exp=baseline
# Flags to disable all the new changes
!python main.py --exp_name $exp --post --emb_off --cmc_off --aw_off --new_kf_off --grid_off --dataset mot17
!python main.py --exp_name $exp --post --emb_off --cmc_off --aw_off --new_kf_off --grid_off -dataset mot20 --track_thresh 0.4
!python main.py --exp_name $exp --post --emb_off --cmc_off --aw_off --new_kf_off --grid_off --dataset dance --aspect_ratio_thresh 1000

In [ ]:
%exp=best_paper_ablations
!python main.py --exp_name $exp --post --grid_off --new_kf_off --dataset mot17 --w_assoc_emb 0.75 --aw_param 0.5
!python main.py --exp_name $exp --post --grid_off --new_kf_off --dataset mot20 --track_thresh 0.4 --w_assoc_emb 0.75 --aw_param 0.5
!python main.py --exp_name $exp --post --grid_off --new_kf_off --dataset dance --aspect_ratio_thresh 1000 --w_assoc_emb 1.25 --aw_param 1

In [10]:
!python external/TrackEval/scripts/run_mot_challenge.py \
  --SPLIT_TO_EVAL val \
  --METRICS HOTA MOTA Identity \
  --TRACKERS_TO_EVAL sa_aw_ablations \
  --GT_FOLDER results/gt/ \
  --TRACKERS_FOLDER results/trackers/ \
  --BENCHMARK MOT17


Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 8                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : /home/Deep-OC-SORT/external/TrackEval/error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : False                         
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          

MotChallenge2DBox Config:
PRINT_CONFIG         : True                          
GT_FOLDER            : results/gt/                   
TRACKERS_FOLDER      : results/trackers/             
OUTPUT_FOLDER      

### Test

In [ ]:
!python main.py --exp_name DeepOC_mot17-test --dataset mot17 --test_dataset

--

In [6]:
import numpy as np

matrix = np.array([[0, 2, 3],
                   [2, 0, 6],
                   [3, 6, 0]])
matrix2 = np.expand_dims(np.max(matrix, axis=1), axis=0) 
print(matrix2.shape)
print(matrix2)

matrix2 = matrix2.repeat(4, axis=0)
print(matrix2.shape)
print(matrix2)

(1, 3)
[[3 6 6]]
(4, 3)
[[3 6 6]
 [3 6 6]
 [3 6 6]
 [3 6 6]]
